All code in this notebook is from https://github.com/kwea123/nerf_pl

# Installation

In [ ]:
!git clone --recursive https://github.com/kwea123/nerf_pl

%cd /content/nerf_pl
!pip install -r requirements.txt

%cd /content/nerf_pl/torchsearchsorted
!pip install .

fatal: destination path 'nerf_pl' already exists and is not an empty directory.
/content/nerf_pl
/content/nerf_pl/torchsearchsorted
Processing /content/nerf_pl/torchsearchsorted
  Created wheel for torchsearchsorted: filename=torchsearchsorted-1.1-cp37-cp37m-linux_x86_64.whl size=3208331 sha256=f4eccbdc60ff9df4f8881a9f8325367f39f150ff5f61c3b1f95f66535484195b
  Stored in directory: /tmp/pip-ephem-wheel-cache-1lqa_nft/wheels/59/fc/44/760e2635d529922fac28c4eb8790e8f6c898fffd1921cf821a
Successfully built torchsearchsorted
  Found existing installation: torchsearchsorted 1.1
    Uninstalling torchsearchsorted-1.1:
      Successfully uninstalled torchsearchsorted-1.1


# Mount your drive (to access data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%cd /content/nerf_pl

import torch
from collections import defaultdict
import numpy as np
import mcubes
import trimesh

from models.rendering import *
from models.nerf import *

from datasets import dataset_dict

from utils import load_ckpt

/content/nerf_pl


# Load model

In [ ]:
# Change here #
img_wh = (1080, 1080) # full resolution of the input images
dataset_name = 'llff' # blender or llff (own data)
scene_name = 'insect' # whatever you want
root_dir = '/content/drive/My Drive/nerf/cc_meshroom' # the folder containing data (images/ and poses_bounds.npy)
# ckpt_path = '/content/drive/My Drive/nerf/cc540/epoch=7.ckpt' # the model path
ckpt_path = '/content/epoch=4.ckpt'
###############

kwargs = {'root_dir': root_dir,
          'img_wh': img_wh}
if dataset_name == 'llff':
    kwargs['spheric_poses'] = True
    kwargs['split'] = 'test'
else:
    kwargs['split'] = 'train'
    
chunk = 1024*32
# dataset = dataset_dict[dataset_name](**kwargs)

embedding_xyz = Embedding(3, 10)
embedding_dir = Embedding(3, 4)

nerf_fine = NeRF()
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')
nerf_fine.cuda().eval();

# Load Blender Model

In [ ]:
# Change here #
img_wh = (1080, 1080) # full resolution of the input images
dataset_name = 'blender' # blender or llff (own data)
scene_name = 'mic' # whatever you want
root_dir = '/content/drive/My Drive/nerf/spaceship_train_val' # the folder containing data (images/ and poses_bounds.npy)
ckpt_path = '/content/epoch=3.ckpt' # the model path
###############

kwargs = {'root_dir': root_dir,
          'img_wh': img_wh}
if dataset_name == 'llff':
    kwargs['spheric_poses'] = True
    kwargs['split'] = 'test'
else:
    kwargs['split'] = 'train'
    
chunk = 1024*32
print(kwargs)
# dataset = dataset_dict[dataset_name](**kwargs)

embedding_xyz = Embedding(3, 10)
embedding_dir = Embedding(3, 4)

nerf_fine = NeRF()
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')
nerf_fine.cuda().eval();

{'root_dir': '/content/drive/My Drive/nerf/spaceship_train_val', 'img_wh': (1080, 1080), 'split': 'train'}


# Search for tight bounds (trial and error!)

In [ ]:
### Tune these parameters until the whole object lies tightly in range with little noise ###
N = 128 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -2, 2 # left/right range
ymin, ymax = -2, 2 # forward/backward range
zmin, zmax = -2, 2 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 1. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
############################################################################################

x = np.linspace(xmin, xmax, N)
y = np.linspace(ymin, ymax, N)
z = np.linspace(zmin, zmax, N)

xyz_ = torch.FloatTensor(np.stack(np.meshgrid(x, y, z), -1).reshape(-1, 3)).cuda()
dir_ = torch.zeros_like(xyz_).cuda()

with torch.no_grad():
    B = xyz_.shape[0]
    out_chunks = []
    for i in range(0, B, chunk):
        xyz_embedded = embedding_xyz(xyz_[i:i+chunk]) # (N, embed_xyz_channels)
        dir_embedded = embedding_dir(dir_[i:i+chunk]) # (N, embed_dir_channels)
        xyzdir_embedded = torch.cat([xyz_embedded, dir_embedded], 1)
        out_chunks += [nerf_fine(xyzdir_embedded)]
    rgbsigma = torch.cat(out_chunks, 0)
    
sigma = rgbsigma[:, -1].cpu().numpy()
sigma = np.maximum(sigma, 0)
sigma = sigma.reshape(N, N, N)

vertices, triangles = mcubes.marching_cubes(sigma, sigma_threshold)

mesh = trimesh.Trimesh(vertices/N, triangles)

mesh.show()

ValueError: ignored

In [ ]:
# You can already export "colorless" mesh if you don't need color
# mcubes.export_mesh(vertices, triangles, f"{scene_name}.dae")
mcubes.export_mesh(vertices, triangles, f"mb.dae")

# Extract colored mesh
Once you find the best range, now **RESTART** the notebook(Runtime->Restart runtime), and copy the configs to the following cell and execute it.

In [ ]:
%cd /content/nerf_pl

# Copy the variables you have above here! ####

# img_wh = (1080, 1080) # full resolution of the input images
# dataset_name = 'blender' # blender or llff (own data)
# scene_name = 'mb' # whatever you want
# root_dir = '/content/drive/My Drive/other_insect/mb' # the folder containing data (images/ and poses_bounds.npy)
# ckpt_path = '/content/epoch=30.ckpt' # the model path


N = 635 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -1.2, 1 # left/right range
ymin, ymax = -1.2, 1 # forward/backward range
zmin, zmax = -1.2, 1 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 10.  # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
###############################################

import os
os.environ['ROOT_DIR'] = root_dir
os.environ['DATASET_NAME'] = dataset_name
os.environ['SCENE_NAME'] = scene_name
os.environ['IMG_SIZE'] = f"{img_wh[0]} {img_wh[1]}"
os.environ['CKPT_PATH'] = ckpt_path
os.environ['N_GRID'] = "635" # final resolution. You can set this number high to preserve more details
os.environ['X_RANGE'] = f"{xmin} {xmax}"
os.environ['Y_RANGE'] = f"{ymin} {ymax}"
os.environ['Z_RANGE'] = f"{zmin} {zmax}"
os.environ['SIGMA_THRESHOLD'] = str(sigma_threshold)
os.environ['OCC_THRESHOLD'] = "0.1" # probably doesn't require tuning. If you find the color is not close
                                    # to real, try to set this number smaller 

!python extract_color_mesh.py \
    --root_dir "$ROOT_DIR" \
    --dataset_name $DATASET_NAME \
    --scene_name $SCENE_NAME \
    --img_wh $IMG_SIZE \
    --ckpt_path $CKPT_PATH \
    --N_grid $N_GRID \
    --x_range $X_RANGE \
    --y_range $Y_RANGE \
    --z_range $Z_RANGE \
    --sigma_threshold $SIGMA_THRESHOLD \
    --occ_threshold $OCC_THRESHOLD

/content/nerf_pl
usage: extract_color_mesh.py [-h] [--root_dir ROOT_DIR]
                             [--dataset_name {blender,llff}]
                             [--scene_name SCENE_NAME]
                             [--img_wh IMG_WH [IMG_WH ...]]
                             [--N_samples N_SAMPLES] [--chunk CHUNK]
                             --ckpt_path CKPT_PATH [--N_grid N_GRID]
                             [--x_range X_RANGE [X_RANGE ...]]
                             [--y_range Y_RANGE [Y_RANGE ...]]
                             [--z_range Z_RANGE [Z_RANGE ...]]
                             [--sigma_threshold SIGMA_THRESHOLD]
                             [--occ_threshold OCC_THRESHOLD]
                             [--use_vertex_normal]
                             [--N_importance N_IMPORTANCE] [--near_t NEAR_T]
extract_color_mesh.py: error: unrecognized arguments: Drive/nerf/cc/epoch=17.ckpt
